**TRAINING PART**

In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

*Prepare and process data*

In [9]:
def load_and_prepare_data(data):
    train_df, test_df = train_test_split(data, test_size=0.1)
    return train_df, test_df

In [10]:
def preprocess_data(df, tokenizer):
    input_texts = ["hỏi: " + question for question in df['question']]
    target_texts = ["trả lời: " + answer for answer in df['answer']]
    model_inputs = tokenizer(input_texts, max_length=512, padding="max_length", truncation=True)

    # Prepare labels for the model
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_texts, max_length=128, padding="max_length", truncation=True)["input_ids"]

    return {
        "input_ids": model_inputs["input_ids"],
        "attention_mask": model_inputs["attention_mask"],
        "labels": labels
    }

In [11]:
class MedicalQADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.encodings['labels'][idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [12]:
def train(model, train_dataset, device, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in DataLoader(train_dataset, batch_size=8, shuffle=True):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item()
        print(f"Epoch {epoch+1}: Loss {total_loss / len(train_dataset)}")

*Hugging face data*

In [13]:
dataset = load_dataset("hungnm/vietnamese-medical-qa")
data_hug = dataset['train'].to_pandas()

In [14]:
train_df, test_df = load_and_prepare_data(data_hug)

*Crawled data*

In [15]:
file_path = 'edoctor_qna.csv'
crawled_data = pd.read_csv(file_path)

In [16]:
crawled_train_df, crawled_test_df = load_and_prepare_data(crawled_data)

In [2]:
if torch.cuda.is_available():       
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [3]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")

In [4]:
model.load_state_dict(torch.load('model_medical_supporter.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

*Hugging face*

In [21]:
train_encoded = preprocess_data(train_df, tokenizer)
train_dataset = MedicalQADataset(train_encoded)

C:\Users\ADMINS\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


*Crawled data*

In [22]:
crawled_train_encoded = preprocess_data(crawled_train_df, tokenizer)
crawled_train_dataset = MedicalQADataset(crawled_train_encoded)

In [5]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [24]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

*Train with hugging face datasets*

In [ ]:
train(model, train_dataset, device, optimizer)

Epoch 1: Loss 0.18553266175062
Epoch 2: Loss 0.17358468127611096
Epoch 3: Loss 0.16251654106156482
Epoch 4: Loss 0.15156945305599398
Epoch 5: Loss 0.14128265080317445
Epoch 6: Loss 0.13153272425897092


*Train with crawled datasets*

In [ ]:
train(model, crawled_train_dataset, device, optimizer)

In [ ]:
torch.save(model.state_dict(), 'model_medical_supporter.pth')

In [ ]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink('model_medical_supporter.pth')

/kaggle/working


/kaggle/working/medical_supporter.pth

In [5]:
def generate_answer(question, model, tokenizer, device):
    model.eval()
    input_text = "hỏi: " + question
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, num_beams=4, early_stopping=True)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [7]:
question = "Tôi tên là gì?"
answer = generate_answer(question, model, tokenizer, device)
print(f"Câu hỏi: {question}")
print(f"Câu trả lời: {answer}")

Câu hỏi: Tôi tên là gì?
Câu trả lời: trả lời: Chào bạn! Xin được trả lời câu hỏi của bạn như sau, với những thông tin mà bạn cung cấp chưa đủ để bác sĩ có thể hỗ trợ tư vấn cho bạn như sau: Bạn có thể chat với bác sĩ edoctor để được hỗ trợ. Chúc bạn nhiều sức khỏe!


In [9]:
question = "Những triệu chứng của sùi mào gà?"
answer = generate_answer(question, model, tokenizer, device)
print(f"Câu hỏi: {question}")
print(f"Câu trả lời: {answer}")

Câu hỏi: Những triệu chứng của sùi mào gà?
Câu trả lời: trả lời: Chào bạn. Bệnh sùi mào gà ở nam là do virus Human papilloma gây nên và thường có thời gian ủ bệnh từ 1-3 tháng với các biểu hiện ban đầu là xuất hiện những u nhú nhỏ, hơi nhô cao trên bề mặt da, có hình tròn hoặc bầu dục, hơi nhô cao trên bề mặt da. Những u nhú này rất to và có kích thước không quá lớn, có màu hồng hoặc trắng đục, hơi nhô cao trên bề mặt da. Những u nhú này rất to và có kích thước không quá lớn, có hình dạng giống như mào gà hay súp lơ. Những u nhú này


**VALIDATION**

In [36]:
questions = []
answers = []

file_path = 'validation_data.csv'

with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for i in range(0, len(lines), 2):
        question = lines[i].strip()
        if i+1 < len(lines):
            answer = lines[i+1].strip()
        else:
            answer = ''
        questions.append(question)
        answers.append(answer)

valid_df = pd.DataFrame({'question': questions, 'answer': answers})
valid_df.head(10)

,question,answer
0,"Thưa bác sĩ. Em đi xét nghiệm máu, bác sĩ xác ...","Chào bạn, có một số lý do vì sao bạn nhiễm HIV..."
1,Cho em hỏi. Năm nay em 18 rồi mà vẫn bị hẹp vớ...,"Chào bạn, tôi hiểu lo lắng của bạn. Về việc hẹ..."
2,Chào bác sĩ ! Tôi bị viêm dạ dày và trào ngược...,"Chào bạn, nếu bạn bị viêm dạ dày và trào ngược..."
3,Chào bác sĩ ! Cách đây 2 năm em có đi cắt bao ...,"Chào bạn, tôi hiểu lo lắng của bạn về tình trạ..."
4,Chào bác sĩ ! Em đã mổ nội soi đường tiết niệu...,"Chào bạn, tôi hiểu lo lắng của bạn. Việc đi ti..."
5,"Bác sĩ cho em hỏi, em nay 16 tuổi, em hút thuố...","Chào bạn, tôi hiểu lo lắng của bạn. Cảm giác đ..."
6,"Chào bác sĩ, em đang băn khoăn về việc dùng qu...","Chào bạn, tôi hiểu lo lắng của bạn. Đúng là th..."
7,"Chào bác sĩ, Bé nhà tôi 10 tuổi mới bị ngộ độc...","Chào bạn, tôi hiểu lo lắng của bạn. Khi bé bị ..."
8,Chào bác sĩ. Cho em xin tham khảo một số bệnh ...,"Chào bạn, tôi hiểu lo lắng của bạn. Bé nhà bạn..."
9,Chào bác sĩ ! Con em sinh ở Từ Dũ được hơn 2 n...,"Chào bạn, tôi hiểu lo lắng của bạn. Con bạn bị..."


In [37]:
def evaluate(model, validation_dataset, device, tokenizer):
    model.eval()
    predictions, references = [], []

    data_loader = DataLoader(validation_dataset, batch_size=8)

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)
            preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            refs = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

            predictions.extend(preds)
            references.extend(refs)

    return predictions, references

In [38]:
valid_encoded = preprocess_data(valid_df, tokenizer)
validation_dataset = MedicalQADataset(valid_encoded)

C:\Users\ADMINS\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [39]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [40]:
predictions, references = evaluate(model, validation_dataset, device, tokenizer)

In [41]:
from rouge_score import rouge_scorer

In [42]:
# Compute ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = {
    'rouge1': {'precision': 0, 'recall': 0, 'fmeasure': 0},
    'rouge2': {'precision': 0, 'recall': 0, 'fmeasure': 0},
    'rougeL': {'precision': 0, 'recall': 0, 'fmeasure': 0},
}

num_samples = len(predictions)
for pred, ref in zip(predictions, references):
    scores = scorer.score(ref, pred)
    for key in rouge_scores.keys():
        rouge_scores[key]['precision'] += scores[key].precision
        rouge_scores[key]['recall'] += scores[key].recall
        rouge_scores[key]['fmeasure'] += scores[key].fmeasure

# Average the scores
for key in rouge_scores.keys():
    rouge_scores[key]['precision'] /= num_samples
    rouge_scores[key]['recall'] /= num_samples
    rouge_scores[key]['fmeasure'] /= num_samples

# Print ROUGE scores
print("ROUGE-1: ", rouge_scores['rouge1'])
print("ROUGE-2: ", rouge_scores['rouge2'])
print("ROUGE-L: ", rouge_scores['rougeL'])

ROUGE-1:  {'precision': 0.6187468492640995, 'recall': 0.6925914652570034, 'fmeasure': 0.629797814699461}
ROUGE-2:  {'precision': 0.2839473119811626, 'recall': 0.3168114291733257, 'fmeasure': 0.28808037531150577}
ROUGE-L:  {'precision': 0.37342785583944427, 'recall': 0.42003194353809503, 'fmeasure': 0.3803065644695269}
